<a href="https://colab.research.google.com/github/dinarsadykow/cv_hw/blob/main/Stepik_SQL_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL stepik
https://stepik.org/course/63054?auth=login

In [1]:
%%capture

import json

# ------------------------------------------------------
# pySpark Tutorial from https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be

!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pickle5
!pip install pyspark_dist_explore

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 --executor-memory 4g --driver-memory 3g pyspark-shell'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

spark = (
      SparkSession.builder\
        .master("local")\
        .appName("sdr")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.maxResultSize", "4g")\
        #.config("spark.sql.crossJoin.enabled", "true")\
         .config("spark.sql.catalogImplementation","hive")
         .config("spark.sql.hive.convertMetastoreParquet","false")
        .getOrCreate()
      )

from pyspark.ml.feature import Tokenizer, HashingTF, IDF

from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import VectorUDT

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

import gc
import re
from datetime import datetime

import pickle5 as pickle
import json

from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

spark

In [40]:
import pandas as pd

t = """('Баранов П.Е.','Москва',700 , '2020-01-12', '2020-01-17'),
('Абрамова К.А.','Владивосток',450 , '2020-01-14', '2020-01-27'),
('Семенов И.В.','Москва',700 , '2020-01-23', '2020-01-31'),
('Ильиных Г.Р.','Владивосток', 450, '2020-01-12', '2020-02-02'),
('Колесов С.П.','Москва',700 , '2020-02-01', '2020-02-06'),
('Баранов П.Е.','Москва', 700, '2020-02-14', '2020-02-22'),
('Абрамова К.А.','Москва', 700, '2020-02-23', '2020-03-01'),
('Лебедев Т.К.','Москва', 700, '2020-03-03', '2020-03-06'),
('Колесов С.П.','Новосибирск',450 , '2020-02-27', '2020-03-12'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-03-29', '2020-04-05'),
('Абрамова К.А.','Москва',700 , '2020-04-06', '2020-04-14'),
('Баранов П.Е.','Новосибирск',450 , '2020-04-18', '2020-05-04'),
('Лебедев Т.К.','Томск',450 , '2020-05-20', '2020-05-31'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-06-01', '2020-06-03'),
('Абрамова К.А.','Санкт-Петербург', 700, '2020-05-28', '2020-06-04'),
('Федорова А.Ю.','Новосибирск',450 , '2020-05-25', '2020-06-04'),
('Колесов С.П.','Новосибирск', 450, '2020-06-03', '2020-06-12'),
('Федорова А.Ю.','Томск', 450, '2020-06-20', '2020-06-26'),
('Абрамова К.А.','Владивосток', 450, '2020-07-02', '2020-07-13'),
('Баранов П.Е.','Воронеж', 450, '2020-07-19', '2020-07-25')"""

df_pd = pd.DataFrame( [ x \
                .replace(')','') \
                .replace('(','') \
                .replace("'",'') \
                .split(',') for x in t.split('\n')] ).drop(5,axis=1)
df_pd.columns = ['name','city','per_diem','date_first','date_last']
df_pd['per_diem'] = df_pd['per_diem'].astype('int32')

df_pd['date_first'] = pd.to_datetime( df_pd['date_first'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')
df_pd['date_last'] = pd.to_datetime( df_pd['date_last'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')

# df_pd


mySchema = StructType([
                       StructField("name", T.StringType(), True)\
                       ,StructField("city", T.StringType(), True)\
                       ,StructField("per_diem", T.IntegerType(), True)\
                       ,StructField("date_first", T.TimestampType(), True)\
                       ,StructField("date_last", T.TimestampType(), True)
                       ])

df_sp = spark \
      .createDataFrame(df_pd ,schema=mySchema ) \
      .withColumn('date_first', F.to_date(F.col('date_first')) ) \
      .withColumn('date_last', F.to_date(F.col('date_last')) )

df_sp.createOrReplaceTempView("trip")

df_sp.printSchema()
df_sp.show()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- per_diem: integer (nullable = true)
 |-- date_first: date (nullable = true)
 |-- date_last: date (nullable = true)

+-------------+---------------+--------+----------+----------+
|         name|           city|per_diem|date_first| date_last|
+-------------+---------------+--------+----------+----------+
| Баранов П.Е.|         Москва|     700|2020-01-12|2020-01-17|
|Абрамова К.А.|    Владивосток|     450|2020-01-14|2020-01-27|
| Семенов И.В.|         Москва|     700|2020-01-23|2020-01-31|
| Ильиных Г.Р.|    Владивосток|     450|2020-01-12|2020-02-02|
| Колесов С.П.|         Москва|     700|2020-02-01|2020-02-06|
| Баранов П.Е.|         Москва|     700|2020-02-14|2020-02-22|
|Абрамова К.А.|         Москва|     700|2020-02-23|2020-03-01|
| Лебедев Т.К.|         Москва|     700|2020-03-03|2020-03-06|
| Колесов С.П.|    Новосибирск|     450|2020-02-27|2020-03-12|
| Семенов И.В.|Санкт-Петербург|     700|2020-0

In [52]:
sql_str = """ select * from trip """
spark.sql(sql_str).toPandas()

,name,city,per_diem,date_first,date_last
0,Баранов П.Е.,Москва,700,2020-01-12,2020-01-17
1,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27
2,Семенов И.В.,Москва,700,2020-01-23,2020-01-31
3,Ильиных Г.Р.,Владивосток,450,2020-01-12,2020-02-02
4,Колесов С.П.,Москва,700,2020-02-01,2020-02-06
5,Баранов П.Е.,Москва,700,2020-02-14,2020-02-22
6,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
7,Лебедев Т.К.,Москва,700,2020-03-03,2020-03-06
8,Колесов С.П.,Новосибирск,450,2020-02-27,2020-03-12
9,Семенов И.В.,Санкт-Петербург,700,2020-03-29,2020-04-05


In [51]:
sql_161 = """SELECT
    name
    , city
    , per_diem
    , date_first
    , date_last

FROM trip

WHERE name LIKE '%а %'
ORDER BY date_last DESC"""

spark.sql(sql_161).toPandas()

,name,city,per_diem,date_first,date_last
0,Абрамова К.А.,Владивосток,450,2020-07-02,2020-07-13
1,Федорова А.Ю.,Томск,450,2020-06-20,2020-06-26
2,Абрамова К.А.,Санкт-Петербург,700,2020-05-28,2020-06-04
3,Федорова А.Ю.,Новосибирск,450,2020-05-25,2020-06-04
4,Абрамова К.А.,Москва,700,2020-04-06,2020-04-14
5,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
6,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27


In [54]:
sql_162 = """SELECT
    name

FROM trip

WHERE city = 'Москва'
ORDER BY name ASC"""

spark.sql(sql_162).toPandas()

,name
0,Абрамова К.А.
1,Абрамова К.А.
2,Баранов П.Е.
3,Баранов П.Е.
4,Колесов С.П.
5,Лебедев Т.К.
6,Семенов И.В.


In [55]:
sql_162 = """SELECT 
    name

FROM trip

WHERE city = 'Москва'
GROUP BY name
ORDER BY name ASC"""

spark.sql(sql_162).toPandas()

,name
0,Абрамова К.А.
1,Баранов П.Е.
2,Колесов С.П.
3,Лебедев Т.К.
4,Семенов И.В.


In [60]:
sql_163 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY city ASC"""

spark.sql(sql_163).toPandas()

,city,Kolichestvo
0,Владивосток,3
1,Воронеж,1
2,Москва,7
3,Новосибирск,4
4,Санкт-Петербург,3
5,Томск,2


In [63]:
sql_164 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY Kolichestvo DESC
LIMIT 2"""

spark.sql(sql_164).toPandas()

,city,Kolichestvo
0,Москва,7
1,Новосибирск,4
